In [1]:
import torch

In [2]:
device = "cpu"

In [3]:
model1 = torch.load("model_anger", map_location=device)

2022-11-05 03:17:10.288223: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-05 03:17:10.578778: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-05 03:17:10.646284: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-05 03:17:10.646315: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [4]:
model2 = torch.load("model_sentiment", map_location=device)

In [9]:
model3 = torch.load("model_emotion", map_location=device)

In [6]:
model3

NameError: name 'model3' is not defined

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [15]:
text = "I am happy"

In [16]:
token = tokenizer.encode_plus(text, return_tensors="pt")

In [13]:
model1(**token)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.8410,  2.5907, -0.0898, -0.8103]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [14]:
model2(**token)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.6538, -0.3212,  3.6224]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [15]:
model3(**token)

SequenceClassifierOutput(loss=None, logits=tensor([[-4.4378, -2.3107, -4.5276, -4.8046,  3.4941, -1.1371,  2.0098, -4.5793,
         -3.6069, -3.2626, -2.3702]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [23]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3

    def forward(self, input_ids, attention_masks, feature1="", feature2=""):

        out1 = self.model1(input_ids=input_ids, attention_mask=attention_masks)
        out2 = self.model2(input_ids=input_ids, attention_mask=attention_masks)
        out3 = self.model3(input_ids=input_ids, attention_mask=attention_masks)

        result = torch.cat([out3["logits"], torch.tensor([[out2["logits"][0][1]]])], 1)

        result[0][0] = out1["logits"][0][0]

        return result

model = BERTClass()
model.to(device);

In [24]:
model(token["input_ids"], token["attention_mask"])

tensor([[-1.8410, -2.3107, -4.5276, -4.8046,  3.4941, -1.1371,  2.0098, -4.5793,
         -3.6069, -3.2626, -2.3702, -0.3212]], grad_fn=<CopySlices>)

In [25]:
torch.save(model, "ensemble")

In [12]:
a = torch.load("ensemble", map_location=device)

In [17]:
a(token["input_ids"], token["attention_mask"])

tensor([[-1.8410, -2.3107, -4.5276, -4.8046,  3.4941, -1.1371,  2.0098, -4.5793,
         -3.6069, -3.2626, -2.3702, -0.3212]], grad_fn=<CopySlices>)

In [18]:
model4 = torch.load("model_emotion_extra", map_location=device)

In [26]:
class BERTClass1(torch.nn.Module):
    def __init__(self):
        super(BERTClass1, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model4

    def forward(self, input_ids, attention_masks, feature1, feature2):

        out1 = self.model1(input_ids=input_ids, attention_mask=attention_masks)
        out2 = self.model2(input_ids=input_ids, attention_mask=attention_masks)
        out3 = self.model3(input_ids=input_ids, attention_mask=attention_masks, feature1=feature1, feature2=feature2)

        result = torch.cat([out3["logits"], torch.tensor([[out2["logits"][0][1]]])], 1)

        result[0][0] = out1["logits"][0][0]

        return result

model = BERTClass1()
model.to(device);

In [28]:
model(token["input_ids"], token["attention_mask"], )

TypeError: forward() missing 2 required positional arguments: 'feature1' and 'feature2'

In [30]:
text

'I am happy'

In [31]:
import spacy
from tqdm import tqdm

2022-11-06 11:16:32.250204: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-06 11:16:32.251627: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-06 11:16:32.252878: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Linux): /proc/driver/nvidia/version does not exist


In [32]:
nlp = spacy.load("en_core_web_sm")

In [33]:
def give_dep_features(sentence):
  sent_dep_list =[]
  doc = nlp(sentence)
  for _token in doc:
    sent_dep_list.append(str(_token.dep_))

  return sent_dep_list

In [34]:
feature1 = give_dep_features(text)
feature1

['nsubj', 'ROOT', 'acomp']

In [36]:
def give_noun_features(sentence):
  sent_dep_list =[]
  doc = nlp(sentence)
  for _token in doc:
    sent_dep_list.append(str(_token.head.pos_))

  return sent_dep_list

In [37]:
feature2 = give_noun_features(text)
feature2

['AUX', 'AUX', 'AUX']

In [40]:
noun_dictionary = {'VERB': 1, 'NOUN': 2, 'AUX': 3, 'ADP': 4, 'PROPN': 5, 'ADJ': 6, 'SYM': 7, 'ADV': 8, 'PRON': 9, 'NUM': 10, 'PUNCT': 11, 'INTJ': 12, 'X': 13, 'SCONJ': 14, 'SPACE': 15, 'PART': 16, 'DET': 17, 'CCONJ': 18}

In [41]:
dependency_dictionary = {'nsubj': 1, 'punct': 2, 'ROOT': 3, 'prep': 4, 'dobj': 5, 'pobj': 6, 'advmod': 7, 'det': 8, 'amod': 9, 'aux': 10, 'compound': 11, 'conj': 12, 'ccomp': 13, 'cc': 14, 'dep': 15, 'nmod': 16, 'poss': 17, 'npadvmod': 18, 'advcl': 19, 'acomp': 20, 'neg': 21, 'attr': 22, 'xcomp': 23, 'appos': 24, 'mark': 25, 'relcl': 26, 'nummod': 27, 'prt': 28, 'auxpass': 29, 'intj': 30, 'pcomp': 31, 'acl': 32, 'nsubjpass': 33, 'case': 34, 'dative': 35, 'quantmod': 36, 'csubj': 37, 'oprd': 38, 'predet': 39, 'agent': 40, 'expl': 41, 'parataxis': 42, 'meta': 43, 'preconj': 44, 'csubjpass': 45}

In [50]:
def give_padding(sent):

  var_len = len(sent)
  padding_len = 110-var_len
  padding = [0]*padding_len
  sent = sent + padding

  return sent[:110]

def give_encoded_noun_list(sent):
  encoded_list = []

  for i in sent:
    encoded_list.append(noun_dictionary[i])

  return give_padding(encoded_list)

def give_encoded_dependency_list(sent):
  encoded_list = []

  for i in sent:
    encoded_list.append(dependency_dictionary[i])

  return give_padding(encoded_list)

In [55]:
print(give_encoded_dependency_list(feature1))

[1, 3, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [53]:
print(give_encoded_noun_list(feature2))

[3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
